In [2]:
# Carga de librerias
from IPython.display import display
import subprocess as sp
import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
from sklearn.metrics import r2_score
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams
import random
import time

# Configurar parametros de pandas y matplotlib
pd.set_option('display.precision', 4)
%config IPCompleter.greedy=True
%matplotlib inline

params = {}
plt.rcParams.update(params)
params = {
           'legend.fontsize': 20,
           'legend.handlelength': 2,
           'axes.titlesize':'20',
           'xtick.labelsize':'20',
           'ytick.labelsize':'20'
         }

matplotlib.rcParams.update(params)

In [7]:
nombre_estaciones = ["belen", "poblado", "aranjuez"]
cod_est_calidad = ["83", "84", "86"]
ruta_calidad = r'data/raw/calidad/{}/'

def concatenar_datos_calidad(ruta_csv, estacion):
    # Generar lista de archivos CSV en la carpeta indicada
    archivos_csv = glob.glob(ruta_csv.format(estacion) + "*.csv")
    lista_df = []

    if not archivos_csv:
        print(f"No se encontraron archivos CSV en {ruta_csv.format(estacion)}")
        return pd.DataFrame()

    for archivo in archivos_csv:
        try:
            # Leer cada archivo CSV y agregarlo a la lista de DataFrames
            _df = pd.read_csv(archivo, index_col=0, header=0)
            lista_df.append(_df)
        except Exception as e:
            print(f"Error al leer {archivo}: {e}")

    # Verificar si se leyeron DataFrames
    if not lista_df:
        print(f"No se pudo leer ningún DataFrame para la estación {estacion}")
        return pd.DataFrame()

    # Concatenar todos los DataFrames
    df_var = pd.concat(lista_df, axis=0)
    df_var.index.name = "fecha_hora"

    # Eliminar índices duplicados
    return df_var.loc[~df_var.index.duplicated()]

# Crear la carpeta de salida si no existe
os.makedirs("data/concatenado/", exist_ok=True)

# Procesar cada estación y guardar el archivo concatenado
for est, cod_cal in zip(nombre_estaciones, cod_est_calidad):
    df_calidad = concatenar_datos_calidad(ruta_calidad, cod_cal)
    
    # Sólo guardar si se obtuvo un DataFrame con datos
    if not df_calidad.empty:
        salida = os.path.join("data/concatenado", f"{est}_calidad_raw.csv")
        df_calidad.to_csv(salida)
        print(f"Archivo guardado en: {salida}")
    else:
        print(f"Sin datos para la estación {est}")


Archivo guardado en: data/concatenado\belen_calidad_raw.csv
No se encontraron archivos CSV en data/raw/calidad/84/
Sin datos para la estación poblado
Archivo guardado en: data/concatenado\aranjuez_calidad_raw.csv


In [12]:
df_calidad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10992 entries, 2023-07-01 00:00:00 to 2024-09-30 23:00:00
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   codigoSerial          10992 non-null  int64  
 1   pm25                  10992 non-null  float64
 2   calidad_pm25          10992 non-null  float64
 3   pm10                  10992 non-null  float64
 4   calidad_pm10          10992 non-null  float64
 5   pm1                   10992 non-null  float64
 6   calidad_pm1           10992 non-null  float64
 7   no                    10992 non-null  float64
 8   calidad_no            10992 non-null  float64
 9   no2                   10992 non-null  float64
 10  calidad_no2           10992 non-null  float64
 11  nox                   10992 non-null  float64
 12  calidad_nox           10992 non-null  float64
 13  ozono                 10992 non-null  float64
 14  calidad_ozono         10992 non-null  float

In [13]:
conteo_codigos = df_calidad.groupby("codigoSerial").size()
print(conteo_codigos)

codigoSerial
86    10992
dtype: int64


In [ ]:
# Cargar todos los archivos CSV concatenados de la carpeta
archivos_concatenados = glob.glob("data/concatenado/*_calidad_raw.csv")
lista_df = []

for archivo in archivos_concatenados:
    df = pd.read_csv(archivo, index_col=0)
    print(f"Cargando archivo: {archivo}")
    # Mostrar los valores únicos en la columna 'codigo serial' para cada archivo
    print("Valores únicos en 'codigo serial':", df["codigoSerial"].unique())
    lista_df.append(df)

# Concatenar todos los DataFrames en uno solo
df_total = pd.concat(lista_df, axis=0)

print("Conteo de códigos seriales en todos los archivos concatenados:")
print(df_total["codigoSerial"].value_counts())


Cargando archivo: data/concatenado\aranjuez_calidad_raw.csv
Valores únicos en 'codigo serial': [86]
Cargando archivo: data/concatenado\belen_calidad_raw.csv
Valores únicos en 'codigo serial': [83]
Conteo de códigos seriales en todos los archivos concatenados:
codigoSerial
86    10992
83    10992
Name: count, dtype: int64
